In [9]:
from rank_bm25 import BM25Okapi
from nltk.stem import PorterStemmer
import json
from nltk.tokenize import word_tokenize
import numpy as np
# import nltk
# nltk.download('stopwords')

# stop_words = set(nltk.corpus.stopwords.words('english'))
ps = PorterStemmer()

In [10]:
def process_data(data):
    # remove stop words
    # data = [word.lower() for word in data if word not in stop_words]
    # convert to string joined by space
    # data = ' '.join(data)

    # tokenize
    tokenized_words = word_tokenize(data)

    # stemming
    tokenized_words = [ps.stem(word) for word in tokenized_words]
    return tokenized_words

In [11]:
def process_query(query):
    return word_tokenize(query)

In [12]:
symp_details = []
with open('../cleaned_data/symp_cleaned.json') as f:
    symp_details = json.load(f)

# filter out symptoms current field ends with "-1.html"

symp_details = [symp['details'] for symp in symp_details if symp['current'].endswith('-1.html')]
print(np.array(symp_details).shape)
# tokenize the symptoms
symp_details_tokenized = [process_data(symp) for symp in symp_details]
bm25_symp = BM25Okapi(symp_details_tokenized)

(139,)


In [13]:
query = "treament sore throat cough children"

predicted_symp = bm25_symp.get_top_n(process_data(query), symp_details, n=5)

# write to predicted_symp.json
with open('../output/predicted_symp.json', 'w') as f:
    json.dump(predicted_symp, f, indent=4)

In [14]:
with open('../cleaned_data/splm_cleaned.json') as f:
    splms = json.load(f)

splms_details = [str(splm['overview']) + str(splm['uses']) for splm in splms]
# print(np.array(splms_details).shape)
# tokenize the symptoms
splms_tokenized = [process_data(splm) for splm in splms_details]
bm25_splm = BM25Okapi(splms_tokenized)

In [15]:
# Match tu input cua nguoi dung den cac detail cua cac symptoms, sau do dem cac symptoms nay match voi cac splm
predicted_splm = {}
for symp in predicted_symp:
    predicted_splm[symp] = bm25_splm.get_top_n(process_query(symp), splms, n=1)

# write to predicted_splm.json
with open('../output/predicted_splm.json', 'w') as f:
    # loop through the dict and write the values to the file
    res = []
    for key, value in predicted_splm.items():
        res.append(value[0])

    json.dump(res, f, indent=4)

In [16]:
# Khong qua match voi symptoms data, match truc tiep tu input cua nguoi dung toi cac splm
predicted_splm = bm25_splm.get_top_n(process_query(query), splms, n=10)

# write to predicted_splm.json
with open('../output/directly_predicted_splm.json', 'w') as f:
    json.dump(predicted_splm, f, indent=4)